In [1]:
!pip install optuna --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.3 MB/s eta 0:00:00


In [2]:
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from torch.optim import AdamW

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay,accuracy_score
import optuna
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:",device)

Mounted at /content/drive
Using: cuda


In [4]:
# Load data
train_df = pd.read_csv("/content/drive/My Drive/ADL Final Project/yelp_dataset_train.csv")
test_df = pd.read_csv("/content/drive/My Drive/ADL Final Project/yelp_dataset_test.csv")

# Sample 10% for tuning
df_sampled, _ = train_test_split(train_df, test_size = 0.90, stratify = train_df['class_index'], random_state = 42)

# Split sampled data into train/val for early stopping
train_small, val_small = train_test_split(df_sampled, test_size = 0.2, stratify = df_sampled['class_index'], random_state = 42)

In [5]:
# Step 1: Extract text and labels
texts = train_small['review_text'].tolist()
labels = train_small['class_index'].tolist()

# Step 2: Initialize tokenizer
vocab_size = 20000
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(texts)

# Step 3: Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_small['review_text'])
val_sequences = tokenizer.texts_to_sequences(val_small['review_text'])

# Step 4: Pad sequences to a fixed length
max_length = 512

train_padded = pad_sequences(train_sequences, maxlen = max_length, padding = 'post', truncating = 'post')
val_padded = pad_sequences(val_sequences, maxlen = max_length, padding = 'post', truncating = 'post')

In [6]:
y_train = train_small['class_index'].values
y_val = val_small['class_index'].values

In [7]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout):
        super(BiLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, bidirectional = True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # *2 for bidirectional
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        pooled = torch.mean(lstm_out, dim = 1)
        out = self.dropout(pooled)
        out = self.fc(out)
        return self.softmax(out)

In [8]:
# Convert padded sequences and labels to PyTorch tensors

X_train_tensor = torch.tensor(train_padded, dtype = torch.long)
y_train_tensor = torch.tensor(y_train, dtype = torch.long)
X_val_tensor = torch.tensor(val_padded, dtype = torch.long)
y_val_tensor = torch.tensor(y_val, dtype = torch.long)

# Create Dataset objects
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size = 128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 128, shuffle=False)

In [9]:
# Objective function for Optuna
def objective(trial):
    start_time = time.time()

    # Hyperparameters for LSTM units, learning rate, and weight decay
    hidden_dim = trial.suggest_int("lstm_units", 64, 256)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    dropout_rate = 0.3  # Fixed dropout rate

    # Set fixed parameters
    vocab_size = 20000  # Adjust based on your dataset
    embed_dim = 100
    output_dim = 3  # 3 classes: negative, neutral, positive
    num_epochs = 3  # Number of epochs

    # Initialize model
    model = BiLSTMClassifier(vocab_size, embed_dim, hidden_dim, output_dim, dropout_rate).to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        epoch_losses = []
        for batch in train_loader:  # train_loader from your sampled 10% training data
            inputs, targets = batch  # adjust based on your DataLoader output
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
        avg_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Trial {trial.number}, Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

    # Evaluation on validation set
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch in val_loader:  # val_loader from your sampled validation data
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_accuracy = accuracy_score(all_targets, all_preds)
    val_loss = sum(epoch_losses) / len(epoch_losses)  # Average loss on validation
    elapsed_time = time.time() - start_time

    # Capture the trial information
    trial_info = {
        'trial_number': trial.number,
        'learning_rate': learning_rate,
        'weight_decay': weight_decay,
        'lstm_units': hidden_dim,
        'val_accuracy': val_accuracy,
        'val_loss': val_loss,
        'train_time': elapsed_time,
    }

    print(f"Trial {trial.number} - LR: {learning_rate} | WD: {weight_decay} | LSTM Units: {hidden_dim} | "
          f"Loss: {val_loss} | Accuracy: {val_accuracy} | Time: {elapsed_time:.2f}s")

    # Optuna maximizes accuracy
    return val_accuracy

study_path = "/content/drive/My Drive/ADL Final Project/optuna_study.db"

# Create the Optuna study
study = optuna.create_study(storage = f"sqlite:///{study_path}", direction = "maximize")

# Run the optimization
study.optimize(objective, n_trials = 20)

[I 2025-04-14 04:17:04,809] A new study created in RDB with name: no-name-5e4341b8-33e6-49b1-bcf4-25badf2c0fcd


Trial 0, Epoch 1/3, Loss: 1.0746
Trial 0, Epoch 2/3, Loss: 1.0286
Trial 0, Epoch 3/3, Loss: 0.8958


[I 2025-04-14 04:18:52,178] Trial 0 finished with value: 0.6789285714285714 and parameters: {'lstm_units': 232, 'learning_rate': 0.0008462171253028531, 'weight_decay': 3.3675872097409635e-06}. Best is trial 0 with value: 0.6789285714285714.


Trial 0 - LR: 0.0008462171253028531 | WD: 3.3675872097409635e-06 | LSTM Units: 232 | Loss: 0.8958458766737818 | Accuracy: 0.6789285714285714 | Time: 107.29s
Trial 1, Epoch 1/3, Loss: 1.0457
Trial 1, Epoch 2/3, Loss: 0.9696
Trial 1, Epoch 3/3, Loss: 0.8111


[I 2025-04-14 04:21:26,256] Trial 1 finished with value: 0.7582142857142857 and parameters: {'lstm_units': 141, 'learning_rate': 0.001513883127515937, 'weight_decay': 5.710195978844217e-05}. Best is trial 1 with value: 0.7582142857142857.


...

Trial 17 - LR: 0.001636542120469616 | WD: 0.00016795063732367538 | LSTM Units: 95 | Loss: 0.8304977498580295 | Accuracy: 0.7317857142857143 | Time: 26.25s
Trial 18, Epoch 1/3, Loss: 1.0973
Trial 18, Epoch 2/3, Loss: 1.0950
Trial 18, Epoch 3/3, Loss: 1.0868


[I 2025-04-14 04:47:24,715] Trial 18 finished with value: 0.40523809523809523 and parameters: {'lstm_units': 141, 'learning_rate': 4.254422876470329e-05, 'weight_decay': 1.4942873954598515e-05}. Best is trial 13 with value: 0.8002380952380952.


Trial 18 - LR: 4.254422876470329e-05 | WD: 1.4942873954598515e-05 | LSTM Units: 141 | Loss: 1.0867740201406153 | Accuracy: 0.40523809523809523 | Time: 154.05s
Trial 19, Epoch 1/3, Loss: 0.9438
Trial 19, Epoch 2/3, Loss: 0.7915
Trial 19, Epoch 3/3, Loss: 0.7348


[I 2025-04-14 04:47:50,046] Trial 19 finished with value: 0.7904761904761904 and parameters: {'lstm_units': 90, 'learning_rate': 0.004241544346115228, 'weight_decay': 0.00025980889070388466}. Best is trial 13 with value: 0.8002380952380952.


Trial 19 - LR: 0.004241544346115228 | WD: 0.00025980889070388466 | LSTM Units: 90 | Loss: 0.734802420828279 | Accuracy: 0.7904761904761904 | Time: 25.26s


In [15]:
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

Best hyperparameters: {'lstm_units': 64, 'learning_rate': 0.009885647542627366, 'weight_decay': 0.00959695196216962}
